### Data Survey

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\PC\Desktop\Estudio\Analisis de Datos\Proyectos\Festival Purchase Behavior Analysis\Datasets\festival_dataset_dirty.csv")

# This part of the code provides an overview of the dataset, spaced, so each function is easier to read.

print(df.shape)
print("\n")
print(df.isnull().sum())
print("\n")
print(df.duplicated().sum())
print("\n")
print(df.nunique())
print("\n")

# I'm selecting columns of type 'object' (text) or 'category' to analyze unique values in those columns.
text_columns = df.select_dtypes(include=['object', 'category']).columns

# Displaying unique values for each text column
# With it, we can see the unique values in each text column, which helps us understand the dataset better
# And detect typos or inconsistencies in the data.
for col in text_columns:
    print(f"\nUnique values of '{col}':")
    print(df[col].unique())

With this first glimpse we recognise:

1. Columns "gender" and "ticket_type" have null values.
2. No duplicated rows.
3. Unnecesary columns.
4. Typos in columns "payment_method", "favourite_genre", "recommend_to_friend"

### Removing non-essential fields

In [ ]:
df = df.drop(["ticket_id",
         "attendee_id",
         "entry_time", 
         "purchase_date", 
         "was_present", 
         "transport_used",
         "top_artist_seen", 
         "origin_city"], axis=1)

### Null values handling

In [ ]:
# --- Gender column ---
# Null values management

# Count the number of unique values in each column
# Used to understand the dataset better
print(df['gender'].value_counts(dropna=False))
print("\n")

# gender_dist will store the normalized distribution
gender_dist = df["gender"].value_counts(normalize=True)

# mask will be used to locate the null values in the dataset
mask = df["gender"].isnull()

# Adds all the null
n_nulls = mask.sum()

# Fill the null values with random choices based on the distribution
# This will ensure that the null values are filled in a way that reflects the original distribution
df.loc[mask, "gender"] = np.random.choice(
    gender_dist.index,
    size=n_nulls,
    p=gender_dist.values
)

# --- Ticket Type column ---
# Same steps to clean "ticket_type" column as we followed for "gender" column
print(df["ticket_type"].value_counts(dropna=False))
print("---------------------------------")

type_dist = df["ticket_type"].value_counts(normalize=True)
mask = df["ticket_type"].isnull()
n_nulls = mask.sum()
df.loc[mask, "ticket_type"] = np.random.choice(
    type_dist.index,
    size = n_nulls,
    p=type_dist.values
)


print(df["gender"].value_counts(dropna=False))
print("\n")
print(df["ticket_type"].value_counts(dropna=False))

### Typos cleaning

In [ ]:
# --- Typos cleaning ---

df["payment_method"] = df["payment_method"].replace({"cash ": "Cash"})
df["favourite_genre"] = df["favourite_genre"].replace("hiphop", "Hip-Hop")
df["favourite_genre"] = df["favourite_genre"].replace("Regueton", "Reggaeton")
df["recommend_to_friend"] = df["recommend_to_friend"].replace({"nO": "No"})

### Trimming values with spaces

In [ ]:
# Strips leading and trailing whitespace from all string columns
for col in df.columns:
    if df[col].dtype == 'object':
      # It will convert to string, then strip whitespace
        df[col] = df[col].str.strip()
      # It will replace multiple spaces with a single space
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

### Type convertion

In [ ]:
# With it, we ensure that the data types are appropriate for analysis and optimize memory usage
df = df.astype({
    'ticket_type': 'category',
    'ticket_price': 'int',
    'age': 'int',
    'gender': 'category',
    'group_size': 'int',
    'food_expense': 'float',
    'drink_expense': 'float',
    'merch_expense': 'float',
    'payment_method': 'category',
    'favourite_genre': 'category',
    'stages_visited': 'int',
    'satisfaction_score': 'int',
    'security_rating': 'int',
    'cleanliness_rating': 'int',
    'recommend_to_friend': 'bool'
})

# Date conversion
df['attendance_date'] = pd.to_datetime(df['attendance_date'])


In [ ]:
df.info()